In this notebook we explore the metadata obtained using the code get_publication.py only for Jorge BARRERA BALLESTEROS

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from unidecode import unidecode
from urllib.parse import urlencode, quote_plus
import keyring
import requests

In [2]:
invest = pd.read_csv('IA_inves.csv', comment='#')
name1 = []
name2 = []
name3 = []

for i in range(len(invest['last_1'])):
    tmp = f"{invest['last_1'][i]}, {invest['name_1'][i]}"
    name1.append(tmp)
    tmp = f"{invest['last_1'][i]}, {invest['name_1'][i].strip()[0]}."
    name2.append(tmp)

    if invest['last_2'][i] != np.nan:
        tmp = f"{invest['last_1'][i]}-{invest['last_2'][i]}, {invest['name_1'][i].strip()[0]}."
        name3.append(tmp)



In [3]:
token="oWP5e2LVuUCxW9nxbynNEj0YPe2uEtDZCbgiZBej"

# query function
def query(query_field, query_value, fields):
    encoded_query = urlencode({"q": f"{query_field}:{query_value}","fl": f"{fields}","rows": 4000})
    results = requests.get("https://api.adsabs.harvard.edu/v1/search/query?{}".format(encoded_query), headers={'Authorization': 'Bearer ' + token})
    r = results.json()    
    return(r)

In [13]:
r = query('author', name3[6], "bibcode, year, pub, title, keyword, author, aff, citation_count")
data = r['response']["docs"]
n_data = len(data)
print('number of entries:',n_data)

number of entries: 122


In [164]:
print(type(data[0]['title'][0]))

<class 'str'>


In [165]:
for item in data:
    item.update( {"id_research":6})

In [166]:
df = pd.DataFrame.from_dict(data)
# not good idea to use an array with strings that have elements with commas on it, like : 'Barrera-Ballesteros, J.', t
first_author, second_author, third_author  = [],[],[]
first_aff, second_aff, third_aff           = [],[],[]
author_pos = []
title = []
for i in range(n_data):
    first_author.append(data[i]['author'][0].replace(',', '')) 
    # if any(data[i]['author'][1]) == True: 
        # second_author.append(data[i]['author'][1].replace(',', ''))         
    # if any(data[i]['author'][2]) == True: 
        # third_author.append(data[i]['author'][2].replace(',', ''))
    # else:
        # third_author.append('')
    first_aff.append(data[i]['aff'][0].replace(',', ''))
    # second_aff.append(data[i]['aff'][1].replace(',', ''))
    # second_aff.append(data[i]['aff'][1]) 
    # third_aff.append(data[i]['aff'][2])
    # TBD ! author_pos.append()
    title.append(data[i]['title'][0])


df['first_author']=  first_author
# df['first_author'], df['second_author'], df['third_author'] =  first_author, second_author, third_author
# df['first_aff'], df['second_aff'], df['third_aff'] =  first_aff, second_aff, third_aff
df['first_aff']=  first_aff
df['title']=  title


# columnsTitles = ['id_research', 'bibcode', 'year', 'pub', 'title', 'keyword', 'first_author', 'aff', 'citation_count']
columnsTitles = ['id_research', 'bibcode', 'year', 'title', 'author', 'first_author', 'first_aff', 'citation_count']
df = df.reindex(columns=columnsTitles)
df.index += 1

# df.info()
df.to_csv('test.csv', index=True, columns = columnsTitles, sep = ';', quotechar = '"')

# metrics 

In [80]:
df_metrics = pd.DataFrame(columns = ['bibcode', 'years', 'citations', 'reads'])
data_list = []
for i in range(n_data):
    data_list.append(data[i]['bibcode'])
    payload = {"bibcodes": [data_list[i]]}
    results = requests.post("https://api.adsabs.harvard.edu/v1/metrics", \
                       headers={'Authorization': 'Bearer ' + token, 
                                "Content-type": "application/json"}, \
                       data=json.dumps(payload))
    metrics = results.json()
    years     = list(metrics['histograms']['citations']['refereed to refereed'].keys())
    citations = list(metrics['histograms']['citations']['refereed to refereed'].values())
    reads     = list(metrics['histograms']['reads']['all reads'].values())
    new_row = {'bibcode': data_list[i], 'years': years, 'citations': citations, 'reads': reads}
    #df_metrics.loc[len(df_metrics)] = new_row
    df_metrics = pd.concat([df_metrics, pd.DataFrame([new_row])], ignore_index=True)
    print(data_list[i], len(years))
    #df_metrics = df_metrics.append(new_row, ignore_index=True)
columnsTitles = ['bibcode', 'years', 'citations', 'reads']
df_metrics = df_metrics.reindex(columns=columnsTitles)
df_metrics.index += 1

# df.info()
df_metrics.to_csv('test_metrics.csv', index=True, columns = columnsTitles, sep = ';', quotechar = '"')   

2022ApJS..259...35A 4
2023ApJS..267...44A 2
2020ApJS..249....3A 6
2017AJ....154...28B 8
2018ApJS..235...42A 8
2022ApJ...940L...7W 2
2023ApJ...955...92V 2
2017ApJS..233...25A 9
2023ApJ...953...56V 2
2019AJ....158..231W 6
2022ApJS..262...36S 3
2016RMxAA..52..171S 9
2024ApJ...960..126V 2
2018RMxAA..54..217S 7
2019ApJS..240...23A 6
2023ApJ...953L..26R 1
2024MNRAS.52710201E 1
2016RMxAA..52...21S 10
2023arXiv231110573S 2
2024arXiv240113181W 1
2014A&A...563A..49S 11
2017ApJ...846..159B 8
2022NewA...9701895L 3
2023MNRAS.526.5555S 2
2016A&A...594A..36S 9
2020AJ....159..167L 5
2021MNRAS.505L..46E 4
2015A&A...576A.135G 10
2023ApJS..268....3C 2
2023ApJ...952..122G 1
2023RMxAA..59..213B 3
2022MNRAS.514.2298B 2
2014A&A...569A...1W 11
2017A&A...597A..48F 9
2018MNRAS.474.2039E 8
2022RASTI...1....3L 2
2023arXiv230413070S 2
2022MNRAS.512.3436E 3
2021ApJ...923...60V 3
2021MNRAS.508.4844N 4
2013A&A...549A..87H 12
2016MNRAS.463.2513B 8
2018ApJ...860...92L 7
2022ApJ...939...40L 2
2021MNRAS.503.1615S 4
2021R

In [62]:
print(years )
citations

['2023', '2024']


[15, 6]

In [67]:
print(metrics['histograms']['reads'].keys())
# data.keys()

dict_keys(['all reads', 'all reads normalized', 'refereed reads', 'refereed reads normalized'])


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 1 to 121
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_research     121 non-null    int64 
 1   bibcode         121 non-null    object
 2   year            121 non-null    object
 3   title           121 non-null    object
 4   first_author    121 non-null    object
 5   first_aff       121 non-null    object
 6   citation_count  121 non-null    int64 
 7   author          121 non-null    object
dtypes: int64(2), object(6)
memory usage: 7.7+ KB


In [114]:
str(df['author']

"1                      [Abdurro'uf, Accetta, Katherine]\n2                 [Almeida, Andrés, Anderson, Scott F.]\n3             [Ahumada, Romina, Allende Prieto, Carlos]\n4           [Blanton, Michael R., Bershady, Matthew A.]\n5                      [Abolfathi, Bela, Aguado, D. S.]\n                             ...                       \n117              [Henao-O, -J., Lady, Granados, Andrés]\n118       [García-Lorenzo, B., Barrera-Ballesteros, J.]\n119    [Barrera-Ballesteros, J. K., Garcia-Lorenzo, B.]\n120                     [Bekeraite, S., Walcher, C. J.]\n121                           [Wong, Tony, Cao, Yixian]\nName: author, Length: 121, dtype: object"